# Imports necessários e o carro que você gostaria de se basear para recomendar

In [ ]:
# Imports

import os, shutil, csv

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.sparse import csr_matrix
from google_images_downloader import GoogleImagesDownloader
from sklearn.neighbors import NearestNeighbors
from matplotlib.table import table
from unidecode import unidecode


In [ ]:
recommend_like = 'Porsche-Boxster'

# Leitura dos Datasets
O que eu preciso?
1) Um [dataset](https://www.kaggle.com/datasets/ankkur13/edmundsconsumer-car-ratings-and-reviews/data?select=Scraped_Car_Review_mazda.csv) que contém o id do usuário e um rating para um veículo, também com o ID dele 
2) Um [dataset](https://www.kaggle.com/datasets/robsonnovato/car-features) que contém o id do veículo com suas características

In [ ]:
# Funções auxiliares

def ler_csv_para_dataframe(caminho_arquivo_csv):
    with open(caminho_arquivo_csv, 'r', newline='', encoding='utf-8') as arquivo_csv:
        leitor_csv = csv.reader(arquivo_csv)
        linhas = list(leitor_csv)        
    df = pd.DataFrame(linhas[1:], columns=linhas[0])
    return df
    
def renomear_arquivos_na_pasta(pasta, palavra_antiga, palavra_nova):
    for nome_arquivo in os.listdir(pasta):
        caminho_arquivo_antigo = os.path.join(pasta, nome_arquivo)
        if palavra_antiga in nome_arquivo:
            novo_nome_arquivo = nome_arquivo.replace(palavra_antiga, palavra_nova)
            caminho_arquivo_novo = os.path.join(pasta, novo_nome_arquivo)
            os.rename(caminho_arquivo_antigo, caminho_arquivo_novo)
    print("Arquivos renomeados com sucesso!")

def obter_nomes_apos_substring(pasta):
    nomes_extraidos = []
    for nome_arquivo in os.listdir(pasta):
        if "Scraped_Car_Review_" in nome_arquivo:
            parte_nome = nome_arquivo.split("Scraped_Car_Review_")[1].replace(".csv", "")
            nomes_extraidos.append(parte_nome)
    return nomes_extraidos

In [ ]:
# Tratamento dos arquivos nas pastas do dataset baixado (alguns nomes são inconsistentes)

pasta = './Data/'
palavra_antiga = "Reviews"
palavra_nova = "Review"
renomear_arquivos_na_pasta(pasta, palavra_antiga, palavra_nova)

palavra_antiga = "Scrapped"
palavra_nova = "Scraped"
renomear_arquivos_na_pasta(pasta, palavra_antiga, palavra_nova)

In [ ]:
# Upload dos datasets

marcas_de_carro = obter_nomes_apos_substring("./Data/")
todos_df = []

for marca in marcas_de_carro:
    file_path = f'./Data/Scraped_Car_Review_{marca}.csv'
    df = ler_csv_para_dataframe(file_path)
    df = df.drop(["Review_Title", "Review", "Review_Date", ""], axis=1)
    todos_df.append(df)

ratings = pd.concat(todos_df, ignore_index=True)
ratings = ratings.dropna()

# Tratativas

In [ ]:
# 1) Criar um ID unico para cada comprador

author_to_id = {}
for i, author in enumerate(ratings["Author_Name"].unique()):
    author_to_id[author] = i
ratings["userID"] = ratings["Author_Name"].apply(lambda author: author_to_id[author])

In [ ]:
# 2) Algumas tratativas do dataset de usuarios

ratings['Company'] = ratings['Vehicle_Title'].str.split().str[1]
ratings['Vehicle_Title'] = ratings['Vehicle_Title'].str.split().str[2:]
ratings['Vehicle_Title'] = ratings['Vehicle_Title'].str.join(' ')
ratings['Model'] = ratings['Vehicle_Title'].str.split().str[0]
ratings = ratings.drop("Vehicle_Title", axis=1)

ratings['carID'] = ratings['Company'].astype(str) + '-' + ratings['Model'].astype(str)

In [ ]:
# 3) Algumas tratativas do dataset de veiculos

cars = pd.read_csv('cars.csv')
cars['carID'] = cars['Company'].astype(str) + '-' + cars['Model'].astype(str)

substituicoes = {
    'cupe': 'coupe',
    'seda': 'sedan',
    'convertible': 'conversivel',
    'hatch': 'hatchback',
    'pick-up': 'picape',
    'pickup': 'picape',
    'hatchback eletrico': 'hatchback',
    'sports car': 'esportivo',
    'hatchback esportivo': 'hatchback',
    'station wagon': 'wagon',
    'truck': 'minivan',
    'seda de luxo': 'sedan',
    'crossover': 'suv',
    'hibrido': 'sedan',
    'furgao': 'van',
    'cabriolet': 'conversivel',
    'subcompacto': 'compacto',
    'media': 'picape',
    'muscle car': 'esportivo'
}

cars['bodytype'] = cars['bodytype'].str.lower().apply(unidecode).replace(substituicoes)
valores_unicos_bodytype = cars['bodytype'].unique()

mapeamento_numerico = {}
for bodytype in valores_unicos_bodytype:
    index = len(mapeamento_numerico) + 1
    mapeamento_numerico[bodytype] = index
cars['bodytype'] = cars['bodytype'].map(mapeamento_numerico)

# Análises estatísticas

In [ ]:
n_ratings = len(ratings)
n_cars = len(ratings['carID'].unique())
n_users = len(ratings['userID'].unique())

print(f"Numero de ratings: {n_ratings}")
print(f"Numero de veiculos unicos: {n_cars}")
print(f"Numero de avaliadores unicos: {n_users}")
print(f"Rating medio por avaliador: {round(n_ratings/n_users, 2)}")
print(f"Numero medio de rating por veiculo: {round(n_ratings/n_cars, 2)}")

In [ ]:
ratings['Rating'] = ratings['Rating'].astype(float)
mean_rating = ratings.groupby('carID')[['Rating']].mean()
lowest_rated = mean_rating['Rating'].idxmin()
cars.loc[cars['carID'] == lowest_rated]
highest_rated = mean_rating['Rating'].idxmax()
cars.loc[cars['carID'] == highest_rated]
ratings[ratings['carID']==highest_rated]
ratings[ratings['carID']==lowest_rated]

car_stats = ratings.groupby('carID')[['Rating']].agg(['count', 'mean'])
car_stats.columns = car_stats.columns.droplevel()

# Método de recomendação por KNN

In [ ]:
def create_matrix(df):
	N = len(df['userID'].unique())
	M = len(df['carID'].unique())
	user_mapper = dict(zip(np.unique(df["userID"]), list(range(N))))
	car_mapper = dict(zip(np.unique(df["carID"]), list(range(M))))
	user_inv_mapper = dict(zip(list(range(N)), np.unique(df["userID"])))
	car_inv_mapper = dict(zip(list(range(M)), np.unique(df["carID"])))
	user_index = [user_mapper[i] for i in df['userID']]
	car_index = [car_mapper[i] for i in df['carID']]
	X = csr_matrix((df["Rating"], (car_index, user_index)), shape=(M, N))
	return X, user_mapper, car_mapper, user_inv_mapper, car_inv_mapper

In [ ]:
def find_similar_cars(recommend_like, X, k, car_mapper, car_inv_mapper, metric='cosine',  show_distance=False):
	neighbour_ids = []
	car_ind = car_mapper[recommend_like]
	car_vec = X[car_ind]
	k+=1
	kNN = NearestNeighbors(n_neighbors=k, algorithm="brute", metric=metric)
	kNN.fit(X)
	car_vec = car_vec.reshape(1,-1)
	neighbour = kNN.kneighbors(car_vec, return_distance=show_distance)
	for i in range(0,k):
		n = neighbour.item(i)
		neighbour_ids.append(car_inv_mapper[n])
	neighbour_ids.pop(0)
	return neighbour_ids

In [ ]:
X, user_mapper, car_mapper, user_inv_mapper, car_inv_mapper = create_matrix(ratings)
similar_ids = find_similar_cars(recommend_like, X, 3, car_mapper, car_inv_mapper)

carros_recomendados = []

car_titles = dict(zip(cars['carID'], cars['carID']))
car_title = car_titles[recommend_like]
carros_recomendados.append(car_title)

for i in similar_ids:
    carros_recomendados.append(car_titles[i])

# Funções de plotagem dos veículos recomendados

In [ ]:
# Download das imagens

def baixar_imagens(array_de_carros):
    downloader = GoogleImagesDownloader(browser="chrome", show=False, debug=False, quiet=False, disable_safeui=False) 
    for carro in array_de_carros:
        downloader.download(carro, limit=1, destination="./carros-sugeridos", file_format="PNG")
    downloader.close()

In [ ]:
def plotar_carros(array_de_carros):
    fig, axs = plt.subplots(2, 2, figsize=(10, 10))

    for i, ax in enumerate(axs.flatten()):
        carro = array_de_carros[i]
        image_path = f'./carros-sugeridos/{carro}/{carro}_0.png'

        if os.path.exists(image_path):
            image = plt.imread(image_path)
            ax.imshow(image)
            ax.set_title(carro)
            ax.axis('off')
        else:
            ax.axis('off')
            ax.set_title(f'Imagem não encontrada para {carro}')

    plt.show()


In [ ]:
def excluir_pasta(caminho_pasta):
    if os.path.exists(caminho_pasta):
        shutil.rmtree(caminho_pasta)
    else:
        print(f"A pasta {caminho_pasta} não existe.")

In [ ]:
def criar_tabela(ax, dados, headers):
    tabela = table(ax, cellText=dados, colLabels=headers, loc='center')
    tabela.auto_set_font_size(False)
    tabela.set_fontsize(10)
    tabela.scale(4, 1.2)
    for i in range(len(headers)):
        tabela[(0, i)].set_facecolor("#a6a6a6") 
        tabela[(1, i)].set_facecolor("red")
    ax.axis('off')
    return tabela

def apresentar_tabela(lista_de_carros_recomendados):
    headers = ['Carro', 'Cavalos', 'Marchas', 'Preço Médio', 'Bodytype', 'Cilindros', 'km/l', 'Lugares']
    dados = []
    mapeamento_string_para_numerico = {v: k for k, v in mapeamento_numerico.items()}
    for i in lista_de_carros_recomendados:
        carro_recomendado = cars.loc[cars['carID'] == i]
        dados.append([
            i,
            carro_recomendado['Cavalos'].values[0],
            carro_recomendado['Marchas'].values[0],
            f'R${carro_recomendado["Preço médio"].values[0]:.2f}',
            mapeamento_string_para_numerico.get(carro_recomendado['bodytype'].values[0], 'Não encontrado'),
            carro_recomendado['cilindros'].values[0],
            carro_recomendado['km/l'].values[0],
            carro_recomendado['lugares'].values[0],
        ])
    fig, ax = plt.subplots()
    tabela = criar_tabela(ax, dados, headers)
    plt.show()


In [ ]:
def criar_grafico_barras_por_ids(ids):
    instancias_selecionadas = cars[cars['carID'].isin(ids)]
    plt.figure(figsize=(10, 6))
    plt.bar(instancias_selecionadas['carID'], instancias_selecionadas['Preço médio'], color='blue')
    
    plt.xlabel('IDs')
    plt.ylabel('Preço Médio')
    plt.title('Preço Médio por ID')
    
    plt.show()

In [ ]:
def plotagem(lista_de_carros, tem_carro_base=False):
    if tem_carro_base:
        print(f"Se voce gostou de {lista_de_carros[0]}, pode gostar de:")
        for carro in lista_de_carros[1:]:
            print(carro)
    else:
        print(f"Carros recomendados:")
        for carro in lista_de_carros:
            print(carro)
    print("\n\n")
    baixar_imagens(lista_de_carros)
    plotar_carros(lista_de_carros)
    apresentar_tabela(lista_de_carros)
    criar_grafico_barras_por_ids(lista_de_carros)
    excluir_pasta("./carros-sugeridos/")

# Plotar veículos recomendados

In [ ]:
plotagem(carros_recomendados, tem_carro_base=True)